<a href="https://colab.research.google.com/github/mzaoualim/UpWork_Proposals/blob/main/Machine_learning/skin_cancer_detection/skin_cancer_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>